In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, func, desc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import label

In [2]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
Base = declarative_base()

In [4]:
class Flight(Base): 
        __tablename__ = 'flights'   
        id = Column(Integer, primary_key=True)
        origin_city = Column(String)
        dest_city = Column(String)
        origin_state = Column(String)
        dest_state = Column(String)
        passengers = Column(Float)
        mail = Column(Float)
        airline = Column(String)
        def __init__(self,
                     origin_city,
                     origin_state,
                     dest_city,
                     dest_state,
                     passengers,
                     mail,
                     airline):
            self.origin_city = origin_city
            self.origin_state = origin_state
            self.dest_city = dest_city
            self.dest_state = dest_state
            self.passengers = passengers
            self.mail = mail
            self.airline = airline

In [5]:
Base.metadata.create_all(engine)

2018-08-22 02:18:16,547 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-22 02:18:16,551 INFO sqlalchemy.engine.base.Engine ()
2018-08-22 02:18:16,553 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-22 02:18:16,554 INFO sqlalchemy.engine.base.Engine ()
2018-08-22 02:18:16,557 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("flights")
2018-08-22 02:18:16,558 INFO sqlalchemy.engine.base.Engine ()
2018-08-22 02:18:16,561 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE flights (
	id INTEGER NOT NULL, 
	origin_city VARCHAR, 
	dest_city VARCHAR, 
	origin_state VARCHAR, 
	dest_state VARCHAR, 
	passengers FLOAT, 
	mail FLOAT, 
	airline VARCHAR, 
	PRIMARY KEY (id)
)


2018-08-22 02:18:16,563 INFO sqlalchemy.engine.base.Engine ()
2018-08-22 02:18:16,567 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
fl_01 = Flight("OC_1", "OS_1", "DC_1", "DC_1", 1, 10, "A_1")
fl_02 = Flight("OC_1", "OS_1", "DC_1", "DC_1", 2, 9, "A_1")
fl_03 = Flight("OC_2", "OS_2", "DC_1", "DC_1", 3, 8, "A_1")
fl_04 = Flight("OC_3", "OS_2", "DC_1", "DC_1", 4, 7, "A_2")
fl_05 = Flight("OC_3", "OS_2", "DC_1", "DC_1", 5, 6, "A_2")
fl_06 = Flight("OC_2", "OS_2", "DC_1", "DC_1", 6, 5, "A_2")
fl_07 = Flight("OC_1", "OS_3", "DC_1", "DC_2", 7, 4, "A_2")
fl_08 = Flight("OC_2", "OS_3", "DC_1", "DC_2", 8, 3, "A_2")
fl_09 = Flight("OC_1", "OS_3", "DC_1", "DC_2", 9, 2, "A_2")
fl_10 = Flight("OC_3", "OS_3", "DC_1", "DC_2", 10, 1, "A_2")


In [7]:
Session = sessionmaker(bind=engine)
db = Session()

In [8]:
db.add(fl_01)
db.add(fl_02)
db.add(fl_03)
db.add(fl_04)
db.add(fl_05)
db.add(fl_06)
db.add(fl_07)
db.add(fl_08)
db.add(fl_09)
db.add(fl_10)



In [18]:
response = db.query(
    Flight.origin_state,
    Flight.origin_city,
    Flight.dest_city,
    Flight.dest_state,
    label("total_passengers", func.sum(Flight.passengers)),
    label("total_mail", func.sum(Flight.mail)),
).group_by(Flight.origin_state, Flight.origin_city).all()
response

2018-08-22 02:26:29,669 INFO sqlalchemy.engine.base.Engine SELECT flights.origin_state AS flights_origin_state, flights.origin_city AS flights_origin_city, flights.dest_city AS flights_dest_city, flights.dest_state AS flights_dest_state, sum(flights.passengers) AS total_passengers, sum(flights.mail) AS total_mail 
FROM flights GROUP BY flights.origin_state, flights.origin_city
2018-08-22 02:26:29,670 INFO sqlalchemy.engine.base.Engine ()


[('OS_1', 'OC_1', 'DC_1', 'DC_1', 3.0, 19.0),
 ('OS_2', 'OC_2', 'DC_1', 'DC_1', 9.0, 13.0),
 ('OS_2', 'OC_3', 'DC_1', 'DC_1', 9.0, 13.0),
 ('OS_3', 'OC_1', 'DC_1', 'DC_2', 16.0, 6.0),
 ('OS_3', 'OC_2', 'DC_1', 'DC_2', 8.0, 3.0),
 ('OS_3', 'OC_3', 'DC_1', 'DC_2', 10.0, 1.0)]

In [20]:
response = db.query(
    Flight.origin_state,
    Flight.origin_city,
    Flight.dest_city,
    Flight.dest_state,
    Flight.airline,
    label("total_passengers", func.sum(Flight.passengers)),
    label("total_mail", func.sum(Flight.mail)),
).group_by(Flight.origin_state, Flight.origin_city, Flight.airline).all()
response

2018-08-22 02:27:18,181 INFO sqlalchemy.engine.base.Engine SELECT flights.origin_state AS flights_origin_state, flights.origin_city AS flights_origin_city, flights.dest_city AS flights_dest_city, flights.dest_state AS flights_dest_state, flights.airline AS flights_airline, sum(flights.passengers) AS total_passengers, sum(flights.mail) AS total_mail 
FROM flights GROUP BY flights.origin_state, flights.origin_city, flights.airline
2018-08-22 02:27:18,183 INFO sqlalchemy.engine.base.Engine ()


[('OS_1', 'OC_1', 'DC_1', 'DC_1', 'A_1', 3.0, 19.0),
 ('OS_2', 'OC_2', 'DC_1', 'DC_1', 'A_1', 3.0, 8.0),
 ('OS_2', 'OC_2', 'DC_1', 'DC_1', 'A_2', 6.0, 5.0),
 ('OS_2', 'OC_3', 'DC_1', 'DC_1', 'A_2', 9.0, 13.0),
 ('OS_3', 'OC_1', 'DC_1', 'DC_2', 'A_2', 16.0, 6.0),
 ('OS_3', 'OC_2', 'DC_1', 'DC_2', 'A_2', 8.0, 3.0),
 ('OS_3', 'OC_3', 'DC_1', 'DC_2', 'A_2', 10.0, 1.0)]

In [25]:
response = db.query(
    Flight.origin_state,
    Flight.origin_city,
    Flight.dest_city,
    Flight.dest_state,
    label("total_passengers", func.sum(Flight.passengers)),
    label("total_mail", func.sum(Flight.mail)),) \
        .filter(Flight.origin_state == "OS_2") \
        .group_by(Flight.origin_city).all()
response

2018-08-22 02:36:41,206 INFO sqlalchemy.engine.base.Engine SELECT flights.origin_state AS flights_origin_state, flights.origin_city AS flights_origin_city, flights.dest_city AS flights_dest_city, flights.dest_state AS flights_dest_state, sum(flights.passengers) AS total_passengers, sum(flights.mail) AS total_mail 
FROM flights 
WHERE flights.origin_state = ? GROUP BY flights.origin_city
2018-08-22 02:36:41,208 INFO sqlalchemy.engine.base.Engine ('OS_2',)


[('OS_2', 'OC_2', 'DC_1', 'DC_1', 9.0, 13.0),
 ('OS_2', 'OC_3', 'DC_1', 'DC_1', 9.0, 13.0)]

In [31]:
response = db.query(
    Flight.airline,
    label("total_passengers", func.sum(Flight.passengers)),
    label("total_mail", func.sum(Flight.mail)),) \
        .group_by(Flight.airline).all()
response

2018-08-22 02:44:07,262 INFO sqlalchemy.engine.base.Engine SELECT flights.airline AS flights_airline, sum(flights.passengers) AS total_passengers, sum(flights.mail) AS total_mail 
FROM flights GROUP BY flights.airline
2018-08-22 02:44:07,264 INFO sqlalchemy.engine.base.Engine ()


[('A_1', 6.0, 27.0), ('A_2', 49.0, 28.0)]

In [ ]:
response = db.query(
    Flight.origin_state,
    Flight.origin_city,
    Flight.airline,
    label("total_passengers", func.sum(Flight.passengers)),
    label("total_mail", func.sum(Flight.mail)),) \
        .filter(Flight.origin_state == "OS_2") \
        .group_by(Flight.airline).all()
response